# LGBM - Training

Sources
1. [LGBM & Deberta Explained by ZULQARNAIN ALI](https://www.kaggle.com/code/zulqarnainalipk/lgbm-deberta-explained)

## Setup

In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.metrics import cohen_kappa_score
import lightgbm as lgb
import pickle as pkl
import wandb
from lightgbm import log_evaluation, early_stopping
from sklearn.metrics import cohen_kappa_score
from sklearn.model_selection import StratifiedKFold

In [2]:
os.chdir("../../")

In [3]:
from lib.config import config
from lib.paths import Paths
from lib.utils.utils import seed_everything
from lib.criterion.metrics import log_metrics
from lib.model.utils import quadratic_weighted_kappa, qwk_obj, get_score

In [4]:
seed_everything()

### Setting Up WandB

In [5]:
WANDB_PROJECT = "Kaggle_ASE_2.0"
WANDB_NAME = f"LGBM-ASE-8"

In [6]:
wandb.login(key=os.environ.get('WANDB_API_KEY'))
wandb.init(
    project=WANDB_PROJECT,
    name=WANDB_NAME,
    save_code=True,
    job_type="train",
    config=config,
)
config = wandb.config

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: shakleenishfar (laplacesdemon43). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/ishfar/.netrc


## Model Training

### Dataset Creation

In [7]:
df = pd.read_csv(Paths.FEATURE_ENGG_CSV_PATH)
df.shape

(17307, 271)

In [8]:
ids = df.loc[:, "essay_id"]
X, y = df.drop(columns=["score", "essay_id"]), df.loc[:, "score"]
del df

X.shape, y.shape

((17307, 269), (17307,))

### Callbacks

In [9]:
callbacks = [
    log_evaluation(period=25),
    early_stopping(stopping_rounds=75, first_metric_only=True),
]

### Constants

In [10]:
a = config.lgbm_a
b = config.lgbm_b

### Training Loop

In [11]:
def train_loop(X_train, y_train, X_valid, y_valid):
    y_train = y_train.astype(np.float32) - a
    y_valid = y_valid.astype(np.float32) - a

    model = lgb.LGBMRegressor(
        objective=qwk_obj,
        metrics="None",
        learning_rate=0.01,
        max_depth=5,
        num_leaves=10,
        colsample_bytree=0.3,
        reg_alpha=0.7,
        reg_lambda=0.1,
        n_estimators=700,
        random_state=config.random_seed,
        extra_trees=True,
        class_weight="balanced",
        n_jobs=6,
        verbosity=0,
    )

    # Train model
    predictor = model.fit(
        X_train,
        y_train,
        eval_names=["train", "valid"],
        eval_set=[(X_train, y_train), (X_valid, y_valid)],
        eval_metric=quadratic_weighted_kappa,
        callbacks=callbacks,
    )

    return model, predictor

In [12]:
def validate_model(idx, X_valid, y_valid, id_valid, predictor):
    y_valid_int = y_valid.astype(np.int64).to_numpy()
    preds = predictor.predict(X_valid) + a
    preds = preds.clip(1, 6).round().astype(np.int64)

    oof = pd.DataFrame(
        {
            "essay_id": id_valid,
            "score": y_valid_int - 1,
            "pred_score": preds - 1,
        }
    )
    log_metrics(oof, f"Fold {idx}")

    return oof

### Fold Loop

In [13]:
skf = StratifiedKFold(
    n_splits=config.lgbm_n_folds,
    random_state=config.random_seed,
    shuffle=True,
)

In [14]:
oof_df = pd.DataFrame()

for idx, (train_idx, valid_idx) in enumerate(skf.split(X, y)):
    X_train, y_train = X.loc[train_idx], y.loc[train_idx]
    X_valid, y_valid = X.loc[valid_idx], y.loc[valid_idx]
    id_valid = ids.loc[valid_idx].to_numpy()

    model, predictor = train_loop(X_train, y_train, X_valid, y_valid)

    with open(f"output/LGBM/{idx}.pkl", "wb") as file:
        pkl.dump({"predictor": predictor, "model": model}, file)

    oof = validate_model(idx, X_valid, y_valid, id_valid, predictor)
    oof_df = pd.concat([oof_df, oof])

[LightGBM] [Info] Using self-defined objective function
Training until validation scores don't improve for 75 rounds
[25]	train's QWK: 0.452839	valid's QWK: 0.496451
[50]	train's QWK: 0.735156	valid's QWK: 0.750937
[75]	train's QWK: 0.801632	valid's QWK: 0.812707
[100]	train's QWK: 0.836271	valid's QWK: 0.842477
[125]	train's QWK: 0.859042	valid's QWK: 0.867971
[150]	train's QWK: 0.868404	valid's QWK: 0.878394
[175]	train's QWK: 0.872931	valid's QWK: 0.882859
[200]	train's QWK: 0.875448	valid's QWK: 0.88972
[225]	train's QWK: 0.877169	valid's QWK: 0.891534
[250]	train's QWK: 0.880066	valid's QWK: 0.89407
[275]	train's QWK: 0.881486	valid's QWK: 0.895178
[300]	train's QWK: 0.882575	valid's QWK: 0.896309
[325]	train's QWK: 0.883465	valid's QWK: 0.896598
[350]	train's QWK: 0.884038	valid's QWK: 0.896545
[375]	train's QWK: 0.884507	valid's QWK: 0.898265
[400]	train's QWK: 0.88484	valid's QWK: 0.897817
[425]	train's QWK: 0.885596	valid's QWK: 0.897817
Early stopping, best iteration is:
[367

/media/ishfar/New Volume/Studies/Projects/Kaggle-Automated-Essay-Scoring/lib/criterion/metrics.py:34: RuntimeWarning: invalid value encountered in scalar divide
  precision = tp / (tp + fp)


[LightGBM] [Info] Using self-defined objective function
Training until validation scores don't improve for 75 rounds
[25]	train's QWK: 0.44904	valid's QWK: 0.477135
[50]	train's QWK: 0.736198	valid's QWK: 0.738301
[75]	train's QWK: 0.793858	valid's QWK: 0.803595
[100]	train's QWK: 0.83482	valid's QWK: 0.852802
[125]	train's QWK: 0.857801	valid's QWK: 0.880176
[150]	train's QWK: 0.867376	valid's QWK: 0.88421
[175]	train's QWK: 0.871787	valid's QWK: 0.889558
[200]	train's QWK: 0.874713	valid's QWK: 0.889502
[225]	train's QWK: 0.877803	valid's QWK: 0.892845
[250]	train's QWK: 0.880068	valid's QWK: 0.896178
[275]	train's QWK: 0.881619	valid's QWK: 0.897048
[300]	train's QWK: 0.883066	valid's QWK: 0.896992
[325]	train's QWK: 0.883632	valid's QWK: 0.898272
[350]	train's QWK: 0.884387	valid's QWK: 0.901045
[375]	train's QWK: 0.884985	valid's QWK: 0.900458
[400]	train's QWK: 0.885378	valid's QWK: 0.899989
Early stopping, best iteration is:
[342]	train's QWK: 0.884096	valid's QWK: 0.901045
Eval

/media/ishfar/New Volume/Studies/Projects/Kaggle-Automated-Essay-Scoring/lib/criterion/metrics.py:34: RuntimeWarning: invalid value encountered in scalar divide
  precision = tp / (tp + fp)


[LightGBM] [Info] Using self-defined objective function
Training until validation scores don't improve for 75 rounds
[25]	train's QWK: 0.455559	valid's QWK: 0.437364
[50]	train's QWK: 0.73596	valid's QWK: 0.727793
[75]	train's QWK: 0.797992	valid's QWK: 0.795334
[100]	train's QWK: 0.836094	valid's QWK: 0.837104
[125]	train's QWK: 0.858937	valid's QWK: 0.868711
[150]	train's QWK: 0.869393	valid's QWK: 0.873924
[175]	train's QWK: 0.873843	valid's QWK: 0.873495
[200]	train's QWK: 0.875764	valid's QWK: 0.87131
[225]	train's QWK: 0.87774	valid's QWK: 0.87615
[250]	train's QWK: 0.880136	valid's QWK: 0.878138
[275]	train's QWK: 0.881723	valid's QWK: 0.880662
[300]	train's QWK: 0.882956	valid's QWK: 0.88085
[325]	train's QWK: 0.883749	valid's QWK: 0.881389
[350]	train's QWK: 0.884637	valid's QWK: 0.883816
[375]	train's QWK: 0.885437	valid's QWK: 0.88651
[400]	train's QWK: 0.885901	valid's QWK: 0.88724
[425]	train's QWK: 0.886369	valid's QWK: 0.89004
[450]	train's QWK: 0.886616	valid's QWK: 0.8

In [15]:
score = get_score(oof_df["score"], oof_df["pred_score"])
print(f'Overall Score: {score:<.4f}')
wandb.log({"CV/qwk_score": score})

Overall Score: 0.8856


In [16]:
wandb.log({"oof_table": wandb.Table(dataframe=oof_df)})

In [17]:
log_metrics(
    pd.DataFrame(
        {
            "score": oof_df.score.to_numpy(),
            "pred_score": oof_df.pred_score.to_numpy(),
        }
    ),
    "Overall",
)

In [18]:
wandb.finish()

CV/qwk_score,▁
CV/qwk_score,0.88558
